In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import time
from keras.utils import np_utils
DATADIR = "C:/Users/Muhammad Zubair Khan/Downloads/Compressed/ddb1_fundusimages/project_data"
CATEGORIES = ["Normal", "DR1", "DR2", "DR3"]

img_size = 100
training_data = []

def create_training_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                new_img = cv2.resize(img_array, (img_size, img_size))
                training_data.append([new_img, class_num])
            except Exception as e:
                pass
            
            
create_training_data()

from keras import backend as k
import random
random.shuffle(training_data)

x = []
y = []

for features, label in training_data:
    x.append(features)
    y.append(label)
            
from keras.utils import np_utils
x = np.array(x).reshape(-1, img_size, img_size, 3)
x = x/255.0
y=np_utils.to_categorical(y, 4) 
      
         
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x, y)

from keras.models import Sequential
from keras.layers.core import Dense, Flatten, Dropout, Activation
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.optimizers import SGD
import matplotlib.pyplot as plt
%matplotlib inline
epochs = 50
batch_size = 32
filter_size = 3
pool_size = 2
num_filters = 64

model = Sequential()
model.add(Conv2D(num_filters, (filter_size, filter_size), padding='valid', input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(num_filters, (filter_size, filter_size), padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))
model.add(Dropout(0.25))
model.add(Conv2D(num_filters, (filter_size, filter_size), padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))
model.add(Dropout(0.25))
model.add(Conv2D(num_filters, (filter_size, filter_size), padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(4))
model.add(Activation('softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

from keras.preprocessing.image import ImageDataGenerator
import os
import matplotlib.pyplot as plt
datagen = ImageDataGenerator(
        rotation_range=10.,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.,
        zoom_range=.1,
        horizontal_flip=True,
        vertical_flip=True)

datagen.fit(x_train)


history = model.fit_generator(datagen.flow(x_train, y_train, batch_size=32),
                    steps_per_epoch=len(x_train) / 32, epochs=epochs, validation_data=(x_val, y_val), verbose=1)


def plot_history(value):
    plt.figure()
    plt.xlabel('Epochs')
    plt.ylabel("Loss")
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.legend(['training', 'validation'])
    
    plt.figure()
    plt.xlabel('Epochs')
    plt.ylabel("accuracy")
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.legend(['training', 'validation'])
    plt.show()
    
plot_history(history)
#loss, accuracy =model.evaluate(x_val, y_val, verbose=0)
#print(loss)
#print(accuracy)


Using TensorFlow backend.


Epoch 1/50
6/5 [==============================] - 27s 5s/step - loss: 1.3927 - acc: 0.3326 - val_loss: 1.3795 - val_acc: 0.3871
Epoch 2/50
6/5 [==============================] - 26s 4s/step - loss: 1.3515 - acc: 0.3912 - val_loss: 1.3466 - val_acc: 0.3871
Epoch 3/50
6/5 [==============================] - 26s 4s/step - loss: 1.3430 - acc: 0.4080 - val_loss: 1.3646 - val_acc: 0.3871
Epoch 4/50
6/5 [==============================] - 26s 4s/step - loss: 1.3316 - acc: 0.4153 - val_loss: 1.3201 - val_acc: 0.3871
Epoch 5/50
6/5 [==============================] - 26s 4s/step - loss: 1.3130 - acc: 0.4122 - val_loss: 1.2933 - val_acc: 0.4516
Epoch 6/50
6/5 [==============================] - 26s 4s/step - loss: 1.2427 - acc: 0.4195 - val_loss: 1.2199 - val_acc: 0.5000
Epoch 7/50
6/5 [==============================] - 26s 4s/step - loss: 1.2450 - acc: 0.4111 - val_loss: 1.1997 - val_acc: 0.4194
Epoch 8/50
6/5 [==============================] - 26s 4s/step - loss: 1.2111 - acc: 0.4478 - val_loss: 1

In [3]:
from sklearn.metrics import classification_report, confusion_matrix
y_predict = model.predict(x_val)
import numpy as np
y_pred = model.predict_classes(x_val)
target_names = ['class 0(Normal)', 'class 1(DR1)', 'class 2(DR2)', 'Class 3(DR3)']
print(classification_report(np.argmax(y_val, axis=1), y_pred, target_names=target_names))
print(confusion_matrix(np.argmax(y_val, axis=1), y_pred))


                 precision    recall  f1-score   support

class 0(Normal)       0.67      0.92      0.77        13
   class 1(DR1)       0.73      0.67      0.70        12
   class 2(DR2)       0.58      0.79      0.67        24
   Class 3(DR3)       0.00      0.00      0.00        13

      micro avg       0.63      0.63      0.63        62
      macro avg       0.49      0.60      0.53        62
   weighted avg       0.50      0.63      0.56        62

[[12  1  0  0]
 [ 2  8  2  0]
 [ 3  2 19  0]
 [ 1  0 12  0]]
